In [1]:
#imports
import pybaseball
from pybaseball import statcast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#vizi settings
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize']= [10,5]
plt.rcParams['font.family'] = 'serif'
plt.style.use('dark_background')
pybaseball.cache.enable()
%matplotlib inline

In [2]:
# collect Statcast data from pitches from the start of 2024 season to the current day
#make the current day dynamic
from datetime import date
today = date.today()
df = statcast(start_dt='2024-03-28', end_dt=str(today))

This is a large query, it may take a moment to complete


100%|██████████| 9/9 [00:51<00:00,  5.75s/it]


In [3]:
#create column that is a flag that shows if a pitch resulted in an out or not
df['IsOutPitch'] = df['events'].isin(['field_out', 'strikeout', 'grounded_into_double_play', 'fielders_choice_out', 'force_out', 'sac_fly', 
                                'sac_bunt', 'strikeout_double_play', 'double_play', 'sac_fly_double_play', 'other_out', 'triple_play', 'sac_bunt_double_play']).astype(bool)

In [4]:
# group the data by pitch type and player name and count the number of out pitches and total pitches for each group
df = df.groupby(['pitch_type', 'player_name'])['IsOutPitch'].agg([('out_pitch_count', 'sum'), ('total_pitches', 'count')]).reset_index()

# calculate the out percentage for each group
df['out_percentage'] = (df['out_pitch_count'] / df['total_pitches'] * 100).round(2)

# filter the dataframe to only include groups with at least min_pitches total pitches
df = df[df['total_pitches'] >= df['total_pitches'].mean()]

# sort the dataframe by out percentage in descending order
df = df.sort_values(by='out_percentage', ascending=False)


In [5]:
import gspread

gc = gspread.service_account(filename='filename.json')

# Authenticate and open the Google Sheet
sheet = gc.open('OutPercentage2024').sheet1

# Clear the existing data in the sheet
sheet.clear()

# Get the data from the pandas dataframe with column headers
data = df.to_numpy().tolist()
data.insert(0, df.columns.to_list())

# Update the Google Sheet with the new data
sheet.update('A1', data)

{'spreadsheetId': '1Lm9sD5HI6_0pZrWCgfu1S_1wSYHWpxIy61T205MgTp4',
 'updatedRange': 'Sheet1!A1:E665',
 'updatedRows': 665,
 'updatedColumns': 5,
 'updatedCells': 3325}